In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import shuffle
from keras.utils import to_categorical
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential
from sklearn.preprocessing import scale
import tensorflow as tf
from keras.layers import Dense
import pickle
from sklearn.metrics import accuracy_score

Read data from excel files

In [2]:
train=pd.read_excel("train.xlsx")
puppies=pd.read_excel("puppies.xlsx")

jOIN THE DATA.

In [3]:
df=train.join(puppies.set_index('PuppyNumber'),on="PuppyNumber",how='left',lsuffix='', rsuffix='_other')
df.drop(columns=["PuppyNumber"],inplace=True)
df.rename(columns=lambda x: x.strip(), inplace=True)
df['breed']=df['breed'].apply(lambda x: x.strip())

DROP COLUMNS WHICH ARE  LESS SIGNIFICANT

In [4]:
df.head()

,Day,Weight,Health,Adult weight,Number of litters,Average number of puppies per litter,Total number of puppies,breed,sex,bith weight,Parent Age
0,1,525.168006,G,28.3,2,9,18,German Shepherd,M,445,1.5
1,1,584.848852,M,28.3,2,9,18,German Shepherd,M,445,1.5
2,1,632.918388,V,28.3,2,9,18,German Shepherd,M,423,1.5
3,1,350.110113,M,28.3,2,9,18,German Shepherd,M,392,1.5
4,1,560.672904,M,28.3,2,9,18,German Shepherd,M,443,1.5


In [5]:
df['breed']=df['breed'].astype('category')
df['breed_codes']=df['breed'].cat.codes
df['sex']=df['sex'].astype('category')
df['sex_codes']=df['sex'].cat.codes

df['Health']=df['Health'].astype('category')
df['Health_codes']=df['Health'].cat.codes

In [6]:
df.dtypes

Day                                        int64
Weight                                   float64
Health                                  category
Adult weight                             float64
Number of litters                          int64
Average number of puppies per litter       int64
Total number of puppies                    int64
breed                                   category
sex                                     category
bith weight                                int64
Parent Age                               float64
breed_codes                                 int8
sex_codes                                   int8
Health_codes                                int8
dtype: object

In [ ]:
numerical_cols=df.select_dtypes(include=['int', 'float','int8'])
numerical_cols.corr()

In [8]:
numerical_cols.corrwith(df['Health_codes'])

Day                                     0.029768
Weight                                  0.021706
Adult weight                           -0.004023
Number of litters                       0.000355
Average number of puppies per litter   -0.003742
Total number of puppies                -0.002049
bith weight                            -0.014244
Parent Age                              0.020072
breed_codes                            -0.000936
sex_codes                               0.001393
Health_codes                            1.000000
dtype: float64

In [9]:
X_data=numerical_cols.drop(columns=['Adult weight',
       'Number of litters', 'Average number of puppies per litter',
       'Total number of puppies','Parent Age','sex_codes','breed_codes','Health_codes'])

In [10]:
X_data.head()

,Day,Weight,bith weight
0,1,525.168006,445
1,1,584.848852,445
2,1,632.918388,423
3,1,350.110113,392
4,1,560.672904,443


In [11]:

# categories = [('sex',['M','F']),('breed',['German Shepherd', 'English Mastiff', 'Maltese'])]
# ohe_columns = [x[0] for x in categories]
# ohe_categories = [x[1] for x in categories]
# encoding= OneHotEncoder(categories=ohe_categories)
# transformer = make_column_transformer((encoding, ohe_columns), remainder='passthrough')

# # We convert the resulting arrays to DataFrames
# dft=transformer.fit_transform(df)
# df=pd.DataFrame(dft,columns=transformer.get_feature_names_out(),index=df.index)
# df.rename(columns={"remainder__Health":"Health"},inplace=True)

Specifying X and Y dataset

In [12]:
with open("health_indicator_categories.pkl",'wb') as f:
    pickle.dump(df['Health'],f)

In [13]:
# X=df.drop(columns=['Health'],axis=1)
X=np.array(X_data)
Y=np.array(df['Health'].cat.codes)

Split data into test and train

In [26]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.1, random_state = 1,shuffle=True)
x_train= np.asarray(x_train).astype('float32')
x_test= np.asarray(x_test).astype('float32')
# x_train.shape, y_train.shape, x_test.shape, y_test.shape

Model Training

In [15]:
best_dct_model=None
best=0

1. Decision Tree Classifier

In [16]:
for i in range (1,20):
    dct_model=DecisionTreeClassifier(splitter='random',criterion="entropy")
    dct_model.fit(x_train,y_train)
    dct_y_pred=dct_model.predict(x_test)
    score =accuracy_score(y_test,dct_y_pred)
    if score>best:
        best_dct_model=dct_model
        best=score

best

0.6975308641975309

2. Random Forest

In [17]:
def try_rf(n):
    random_forest_model=RandomForestClassifier(n_estimators=n,random_state=42)
    random_forest_model.fit(x_train,y_train)
    random_forest_y_pred=random_forest_model.predict(x_test)
    score =accuracy_score(y_test,random_forest_y_pred)
    return random_forest_model,random_forest_y_pred,score

best_score = n = 0
random_forest_best_model = knn_y_pred = None
for i in range(200, 300,20):
    random_forest_model, y_pred, score = try_rf(i)
    if score > best_score:
        best_score = score
        random_forest_best_model = random_forest_model
        random_forest_y_pred = y_pred
        n = i
print("Best Score", best_score, n)

Best Score 0.7160493827160493 280


3.  Logistic Regression

In [18]:
lg_model=LogisticRegression(max_iter=10000)
lg_model.fit(x_train,y_train)
lg_y_pred=lg_model.predict(x_test)
score =accuracy_score(y_test,lg_y_pred)
score

0.345679012345679

4. Support  Vector Machine

In [19]:
svc_model=SVC(max_iter=30000,probability=True)
svc_model.fit(x_train,y_train)
svc_pred=svc_model.predict(x_test)
score =accuracy_score(y_test,svc_pred)
score

0.3395061728395062

5. Naive Bayes

In [20]:
nb_model=GaussianNB()
nb_model.fit(x_train,y_train)
nb_y_pred=nb_model.predict(x_test)
score =accuracy_score(y_test,nb_y_pred)
score

0.35185185185185186

6. Kth Nearest Neighbour

In [21]:
def try_knn(n_neighbors):
    knn_model = KNeighborsClassifier(n_neighbors=n_neighbors,p=5,n_jobs=20,algorithm="kd_tree")
    knn_model.fit(x_train, y_train)
    knn_y_pred = knn_model.predict(x_test)
    score = accuracy_score(y_test, knn_y_pred)
    return knn_model, knn_y_pred, score


best_score = n = 0
knn_best_model = knn_y_pred = None
for i in range(1, 100):
    knn_model, y_pred, score = try_knn(i)
    if score > best_score:
        best_score = score
        knn_best_model = knn_model
        knn_y_pred = y_pred
        n = i
print("Best Score", best_score, n)

Best Score 0.4444444444444444 96


Save the best Models

In [22]:
with open('puppy_management_system_random_forest_model.pkl', 'wb') as file:
    pickle.dump(random_forest_best_model, file)
with open('puppy_management_system_decision_tree_model.pkl', 'wb') as file:
    pickle.dump(best_dct_model, file)

In [23]:
  # Load the saved model
with open('puppy_management_system_random_forest_model.pkl', 'rb') as file:
    loaded_rf_classifier = pickle.load(file)
# Load the saved model
with open('puppy_management_system_decision_tree_model.pkl', 'rb') as file:
    loaded_dct_classifier = pickle.load(file)

rf_y_pred=loaded_rf_classifier.predict(x_test)
rf_score =accuracy_score(y_test,rf_y_pred)

dct_y_pred=loaded_dct_classifier.predict(x_test)
dct_score =accuracy_score(y_test,dct_y_pred)

print(f"RF: {rf_score} DCT: {dct_score}")

RF: 0.7160493827160493 DCT: 0.6975308641975309


In [24]:
d_pred=pd.DataFrame(data={
                        "Day":[1],
                        "Weight":[525.168006],
                        "bith weight":[445]
                          }
                    )
d_pred

,Day,Weight,bith weight
0,1,525.168006,445


In [25]:
loaded_rf_classifier.predict(np.array(d_pred))[0]


0